In [9]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import pandas as pd
from keras.utils.data_utils import get_file
import io

In [2]:
np.random.seed(42)
tf.random.set_seed(42)

In [3]:
shakespeare_url = "https://homl.info/shakespeare"
filepath = keras.utils.get_file("shakespeare.txt", shakespeare_url)
with open(filepath) as f:
    shakespeare_text = f.read()

In [5]:
print('corpus lenght:', len(shakespeare_text))

corpus lenght: 1115394


In [12]:
path = get_file(
    'nietzsche.txt',
    origin='https://s3.amazonaws.com/text-datasets/nietzsche.txt')
with io.open(path, encoding='utf-8') as f:
    text = f.read(200000).lower()
print('corpus length:', len(text))

corpus length: 200000


In [13]:
tokenizer = keras.preprocessing.text.Tokenizer(char_level=True)
tokenizer.fit_on_texts(text)
tokenizer.texts_to_sequences(['First'])

[[15, 5, 9, 8, 3]]

In [14]:
max_id = len(tokenizer.word_index)
dataset_size = tokenizer.document_count

In [15]:
[encoded] = np.array(tokenizer.texts_to_sequences([shakespeare_text])) - 1
train_size = dataset_size * 90 // 100
dataset = tf.data.Dataset.from_tensor_slices(encoded[:train_size])

In [16]:
n_steps = 100
window_length = n_steps + 1
dataset = dataset.repeat().window(window_length, shift=1, drop_remainder=True)

In [17]:
dataset = dataset.flat_map(lambda window: window.batch(window_length))

In [18]:
batch_size = 32
dataset = dataset.shuffle(10000).batch(batch_size)
dataset = dataset.map(lambda windows: (windows[:, :-1], windows[:, 1:]))

In [19]:
dataset = dataset.map(
    lambda X_batch, Y_batch: (tf.one_hot(X_batch, depth=max_id), Y_batch))
dataset = dataset.prefetch(1)

In [20]:
for X_batch, Y_batch in dataset.take(1):
    print(X_batch.shape, Y_batch.shape)

(32, 100, 52) (32, 100)


In [21]:
model = keras.models.Sequential([
    keras.layers.GRU(128, return_sequences=True, input_shape=[None, max_id],
                     dropout=0.2, recurrent_dropout=0.2),
    keras.layers.GRU(128, return_sequences=True,
                     dropout=0.2, recurrent_dropout=0.2),
    keras.layers.TimeDistributed(keras.layers.Dense(max_id,
                                                    activation="softmax"))
])
model.compile(loss="sparse_categorical_crossentropy", optimizer="adam")
history = model.fit(dataset, steps_per_epoch=train_size // batch_size,
                    epochs=10)

Train for 5625 steps
Epoch 1/10
5625/5625 [==============================] - 1748s 311ms/step - loss: 1.6947
Epoch 2/10
  29/5625 [..............................] - ETA: 28:57 - loss: 1.5428

KeyboardInterrupt: 

In [87]:
def preprocess(texts):
    X = np.array(tokenizer.texts_to_sequences(texts)) - 1
    return tf.one_hot(X, max_id)

In [88]:
X_new = preprocess(['When the worl'])
Y_pred = model.predict_classes(X_new)
tokenizer.sequences_to_texts(Y_pred + 1)[0][-1]

';'

In [89]:
def next_char(text, temperature=1):
    X_new = preprocess(text)
    y_proba = model.predict(X_new)[0, -1:, :]
    rescaled_logits = tf.math.log(y_proba) / temperature
    char_id = tf.random.categorical(rescaled_logits, num_samples=1) + 1
    return tokenizer.sequences_to_texts(char_id.numpy())[0]
def complete_text(text, n_chars=50, temperature=1):
    for _ in range(n_chars):
        text += next_char(text, temperature)
    return text

SyntaxError: invalid syntax (<ipython-input-89-3cc2c1a81716>, line 8)

In [ ]:
print(complete_text('t', temperature=0.2))

In [ ]:
print(complete_text('w', temperature=1))

In [ ]:
print(complete_text('w', temperature=2))

## Stateful RNN

In [90]:
dataset = tf.data.Dataset.from_tensor_slices(encoded[:train_size])
dataset = dataset.window(window_lenght, shift=n_steps, drop_remainder=True)
dataset = dataset.flat_map(lambda window: window.batch(window_lenght))
dataset = dataset.batch(1)
dataset = dataset.map(lambda windows: (windows[:, :-1], windows[:,1:]))
dataset = dataset.map(
    lambda X_batch, Y_batch: (tf.one_hot(X_batch, depth=max_id), Y_batch))
dataset = dataset.prefetch(1)

NameError: name 'window_lenght' is not defined

In [91]:
model = keras.models.Sequential([
    keras.layers.GRU(128, return_sequences=True, stateful=True, dropout=0.2, recurrent_dropout=0.2, batch_input_shape=[batch_size, None, max_id]),
    keras.layers.GRU(128, return_sequences=True, stateful=True, dropout=0.2,
    recurrent_dropout= 0.2)
    keras.layers.TimeDistributed(keras.layers.Dense(max_id, activation='softmax'))
])

SyntaxError: invalid syntax (<ipython-input-91-b1bc02e2e55e>, line 5)

In [92]:
class ResetStatesCallback(keras.callbacks.Callback):
    def on_epoch_begin(self, epoch, logs):
        self.model.reset_states()

In [93]:
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam')
model.fit(dataset, epochs=50, callbacks=[ResetStatesCallback()])

ValueError: in converted code:

    D:\miniconda3\envs\myenv\lib\site-packages\tensorflow_core\python\keras\engine\training_v2.py:677 map_fn
        batch_size=None)
    D:\miniconda3\envs\myenv\lib\site-packages\tensorflow_core\python\keras\engine\training.py:2410 _standardize_tensors
        exception_prefix='input')
    D:\miniconda3\envs\myenv\lib\site-packages\tensorflow_core\python\keras\engine\training_utils.py:573 standardize_input_data
        'with shape ' + str(data_shape))

    ValueError: Error when checking input: expected gru_8_input to have 3 dimensions, but got array with shape ()
